# 🔥 Desafio - **Média histórica - Monitoramento hídrico**

## Enunciado:

- As tabelas **`parameter_sample`** e **`parameter_result`** armazenam dados de amostras e resultados, respectivamente, de monitoramento de qualidade hídrica.
- As amostras são derivadas dos pontos de monitoramento (tabela `station`) e os resultados são derivados das amostras. O esquema a seguir ilustra os relacionamentos entre as tabelas.

![station_samples_results](../../img/desafios/aula03/station_samples_result.png)

Duas análises comuns subsidiadas por estes dados são a evolução temporal dos parâmetros (avaliada normalmente por ferramentas estatísticas); e a avaliação de conformidade legal, através da comparação das medições a valores de referência.

## Objetivo

O desafio é escrever uma consulta SQL para identificar resultados de qualidade **acima da média histórica de cada parâmetro em pelo menos 20%**, ou seja, cada resultado deve ser comparado à média de todos os resultados do mesmo parâmetro, no mesmo ponto, anteriormente observados.

Os seguintes campos devem ser retornados pela consulta SQL:
- Codigo HGA: Nome HGA do ponto de monitoramento (station.Name)
- Local: Localização do ponto de monitoramento (station.zlocation)
- ID da amostra: Identificador da amostra de qualidade (parameter_sample.sample_id)
- Data da amostra: Data de amostragem (parameter_sample.sample_date)
- Parametro: Nome do parametro químico analisado (parameter_result.chemical_name)
- Resultado: Valor numérico obtido na análise do parâmetro (parameter_result.result_value)
- Unidade: Unidade referente ao parâmetro analisado (parameter_result.result_unit)
- **Média Histórica: Valor da média do monitoramento do parâmetro**
- **Flag: Identificação se o resultado obtido é, no mínimo, 20% superior à média histórica**

****

In [1]:
-- Dica: Como comparar dinamicamente cada resultado apenas aos anteriores à ele ??

-- Utilizando uma subquery:
SELECT
    s.Name                      AS [Codigo HGA]
    ,s.zlocation                AS [Local]
    ,a.sample_id                AS [ID Amostra]
    ,a.sample_date              AS [Data Amostra]
    ,r.chemical_name            AS [Parametro]
    ,r.result_value             AS [Resultado]
    ,r.result_unit              AS [Unidade]

    ,(
        SELECT AVG(res.result_value)
        FROM station st
            INNER JOIN parameter_sample ams
                ON st.ID = ams.Station
            INNER JOIN parameter_result res
                ON ams.sample_id = res.sample_id
        WHERE
            st.ID = s.ID AND
            res.chemical_name = r.chemical_name AND 
            ams.sample_date <= a.sample_date
    )                           AS [Media Hist]

FROM station s
    INNER JOIN parameter_sample a
        ON s.ID = a.Station
    INNER JOIN parameter_result r
        ON a.sample_id = r.sample_id
WHERE s.Name = 'WST-22' AND r.chemical_name = 'Alcalinidade Total'

(97 rows affected)

Total execution time: 00:00:00.120

Codigo HGA,Local,ID Amostra,Data Amostra,Parametro,Resultado,Unidade,Media Hist
WST-22,Mina D,0417/23-01,2023-01-12 09:51:00.000,Alcalinidade Total,"50,8",mg/L,"63,127777777777766"
WST-22,Mina D,0418/23-01,2023-01-18 09:47:00.000,Alcalinidade Total,"63,5",mg/L,"62,99230769230768"
WST-22,Mina D,0420/23-01,2023-02-03 12:50:00.000,Alcalinidade Total,"31,4",mg/L,"62,997826086956515"
WST-22,Mina D,0483/19-01,2019-01-24 10:20:00.000,Alcalinidade Total,"22,4",mg/L,"41,13636363636364"
WST-22,Mina D,0731/20-01,2020-01-16 09:24:00.000,Alcalinidade Total,"38,3",mg/L,"41,55483870967742"
WST-22,Mina D,0869/22-01,2022-01-24 15:05:00.000,Alcalinidade Total,"68,1",mg/L,"53,69818181818181"
WST-22,Mina D,0904/18-01,2018-01-16 09:35:00.000,Alcalinidade Total,46,mg/L,"40,31"
WST-22,Mina D,10131/22-01,2022-08-11 10:22:00.000,Alcalinidade Total,75,mg/L,"59,481944444444444"
WST-22,Mina D,10132/22-01,2022-08-16 10:29:00.000,Alcalinidade Total,85,mg/L,"59,6945205479452"
WST-22,Mina D,10133/22-01,2022-08-25 12:37:00.000,Alcalinidade Total,79,mg/L,"60,03648648648648"


In [2]:
-- Utilizando a função OVER em conjunto com AVG
SELECT
    s.Name                      AS [Codigo HGA]
    ,s.zlocation                AS [Local]
    ,a.sample_id                AS [ID Amostra]
    ,a.sample_date              AS [Data Amostra]
    ,r.chemical_name            AS [Parametro]
    ,r.result_value             AS [Resultado]
    ,r.result_unit              AS [Unidade]
    ,AVG(r.result_value) OVER (PARTITION BY s.Name, r.chemical_name ORDER BY a.sample_date)  AS [Media Hist]
FROM station s
    INNER JOIN parameter_sample a
        ON s.ID = a.Station
    INNER JOIN parameter_result r
        ON a.sample_id = r.sample_id
WHERE s.Name = 'WST-22' AND r.chemical_name = 'Alcalinidade Total'

(97 rows affected)

Total execution time: 00:00:00.012

Codigo HGA,Local,ID Amostra,Data Amostra,Parametro,Resultado,Unidade,Media Hist
WST-22,Mina D,2180/17-01,2017-03-15 10:11:00.000,Alcalinidade Total,40,mg/L,40
WST-22,Mina D,3189/17-01,2017-04-11 10:39:00.000,Alcalinidade Total,"39,5",mg/L,"39,75"
WST-22,Mina D,4397/17-01,2017-05-09 11:45:00.000,Alcalinidade Total,"46,9",mg/L,"42,13333333333333"
WST-22,Mina D,5463/17-01,2017-06-05 14:40:00.000,Alcalinidade Total,"43,7",mg/L,"42,525000000000006"
WST-22,Mina D,6742/17-01,2017-07-18 10:40:00.000,Alcalinidade Total,45,mg/L,"43,02"
WST-22,Mina D,7677/17-01,2017-08-16 11:04:00.000,Alcalinidade Total,40,mg/L,"42,51666666666667"
WST-22,Mina D,8878/17-01,2017-09-22 11:09:00.000,Alcalinidade Total,50,mg/L,"43,58571428571429"
WST-22,Mina D,9971/17-01,2017-10-17 10:43:00.000,Alcalinidade Total,30,mg/L,"41,8875"
WST-22,Mina D,11031/17-01,2017-11-02 08:56:00.000,Alcalinidade Total,47,mg/L,"42,455555555555556"
WST-22,Mina D,12373/17-01,2017-12-19 10:15:00.000,Alcalinidade Total,21,mg/L,"40,31"


In [3]:
WITH medias AS (
    SELECT
        s.Name                      AS [Codigo HGA]
        ,s.zlocation                AS [Local]
        ,a.sample_id                AS [ID Amostra]
        ,a.sample_date              AS [Data Amostra]
        ,r.chemical_name            AS [Parametro]
        ,r.result_value             AS [Resultado]
        ,r.result_unit              AS [Unidade]
        ,AVG(r.result_value) OVER (PARTITION BY s.Name, r.chemical_name ORDER BY a.sample_date)  AS [Media Hist]
    FROM station s
        INNER JOIN parameter_sample a
            ON s.ID = a.Station
        INNER JOIN parameter_result r
            ON a.sample_id = r.sample_id
    WHERE s.Name = 'WST-22' AND r.chemical_name = 'Alcalinidade Total'
)

SELECT
    *,
    Flag = IIF([Resultado] > 1.2 * [Media Hist], 'Acima', 'Abaixo')
FROM medias


(97 rows affected)

Total execution time: 00:00:00.018

Codigo HGA,Local,ID Amostra,Data Amostra,Parametro,Resultado,Unidade,Media Hist,Flag
WST-22,Mina D,2180/17-01,2017-03-15 10:11:00.000,Alcalinidade Total,40,mg/L,40,Abaixo
WST-22,Mina D,3189/17-01,2017-04-11 10:39:00.000,Alcalinidade Total,"39,5",mg/L,"39,75",Abaixo
WST-22,Mina D,4397/17-01,2017-05-09 11:45:00.000,Alcalinidade Total,"46,9",mg/L,"42,13333333333333",Abaixo
WST-22,Mina D,5463/17-01,2017-06-05 14:40:00.000,Alcalinidade Total,"43,7",mg/L,"42,525000000000006",Abaixo
WST-22,Mina D,6742/17-01,2017-07-18 10:40:00.000,Alcalinidade Total,45,mg/L,"43,02",Abaixo
WST-22,Mina D,7677/17-01,2017-08-16 11:04:00.000,Alcalinidade Total,40,mg/L,"42,51666666666667",Abaixo
WST-22,Mina D,8878/17-01,2017-09-22 11:09:00.000,Alcalinidade Total,50,mg/L,"43,58571428571429",Abaixo
WST-22,Mina D,9971/17-01,2017-10-17 10:43:00.000,Alcalinidade Total,30,mg/L,"41,8875",Abaixo
WST-22,Mina D,11031/17-01,2017-11-02 08:56:00.000,Alcalinidade Total,47,mg/L,"42,455555555555556",Abaixo
WST-22,Mina D,12373/17-01,2017-12-19 10:15:00.000,Alcalinidade Total,21,mg/L,"40,31",Abaixo


# 🤯 Desafio desafiador

Ainda no contexto dos dados de monitoramento de qualidade hídrica, normalmente são utilizados relatórios/gráficos para melhor visualização e análise. Um exemplo de relatório, mostrado a seguir, considera a apresentação dos dados da amostragem em formato tabular e os parâmetros em formato aberto (nas colunas da matriz).

![Report](../../img/desafios/aula03/report.png)

Uma importante análise dos dados de qualidade hídrica se refere à saúde e consistência dos próprios dados (QAQC). Por exemplo, para alguns conjuntos de parâmetros, é esperada uma alta correlação entre os resultados. Um exemplo de comparação ocorre entre os parâmetros **Solidos Dissolvidos Totais (SDT)** e **Condutividade Eletrica**, em que considera-se aceitável uma razão SDT / CE na faixa de 55% - 75%.

## Objetivo

Criar uma consulta SQL para retornar uma análise de QAQC da razão entre os parâmetros **Solidos Dissolvidos Totais (SDT)** e **Condutividade Eletrica**.

O formato da tabela gerada deve ser o seguinte:
- Codigo HGA: Nome HGA do ponto de monitoramento (station.Name)
- ID da amostra: Identificador da amostra de qualidade (parameter_sample.sample_id)
- Solidos Dissolvidos Totais (SDT): Resultados médios do monitoramento de [Solidos Dissolvidos Totais (SDT)] para o ano/mês 
- Condutividade: Resultados médios do monitoramento de [Condutividade Eletrica] para o ano/mês
- SDT / CE: Razão entre o resultado de [Solidos Dissolvidos Totais (SDT)] e o da [Condutividade Eletrica], para análise do QAQC

PS.: O cálculo da razão faz sentido apenas para amostras com resultados **dos dois parâmetros**.

In [4]:
WITH cte AS (
    SELECT
        s.Name                      AS [Codigo HGA]
        ,a.sample_id                AS [ID Amostra]
        ,r.chemical_name            AS [Parametro]
        ,r.result_value             AS [Resultado]
    FROM station s
        INNER JOIN parameter_sample a
            ON s.ID = a.Station
        INNER JOIN parameter_result r
            ON a.sample_id = r.sample_id
    WHERE 
        r.chemical_name IN ('Solidos Dissolvidos Totais (SDT)', 'Condutividade Eletrica')
)

SELECT
    [Codigo HGA]
    ,[ID Amostra]
    ,[Solidos Dissolvidos Totais (SDT)]
    ,[Condutividade Eletrica]
    ,[Razão %] = ROUND([Solidos Dissolvidos Totais (SDT)] / [Condutividade Eletrica] * 100, 2)
FROM cte
PIVOT (
    AVG([Resultado])
    FOR [Parametro] IN ([Solidos Dissolvidos Totais (SDT)] ,[Condutividade Eletrica])
) AS pivot_table
WHERE
    [Solidos Dissolvidos Totais (SDT)] IS NOT NULL
    AND [Condutividade Eletrica] IS NOT NULL

(2935 rows affected)

Total execution time: 00:00:00.088

Codigo HGA,ID Amostra,Solidos Dissolvidos Totais (SDT),Condutividade Eletrica,Razão %
WST-323,0036/17-01,26,"42,16","61,67"
WST-328,0037/17-01,55,"88,97","61,82"
WST-331,0038/17-01,53,"78,8","67,26"
WST-333,0039/17-01,86,"153,28","56,11"
WST-334,0040/17-01,35,"72,35","48,38"
WST-335,0041/17-01,102,"236,4","43,15"
WST-336,0042/17-01,34,"38,92","87,36"
WST-132,0043/17-01,27,"33,84","79,79"
WST-49,0413/23-01,32,"5,4","592,59"
WST-49,0414/23-01,21,"46,2","45,45"
